<a href="https://colab.research.google.com/github/russellemergentai/MistralDocker/blob/main/LangchainMistral7bInstructAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

T4 is fine.

In [3]:
#login
from google.colab import drive
drive.mount('/content/drive')

from huggingface_hub import login
from google.colab import userdata

# Load the secret token
HF_TOKEN = userdata.get('HF_TOKEN')

# Check if the token is set and log in
if HF_TOKEN:
    login(HF_TOKEN)
    print("logged in to Hugging Face")
else:
    print("Hugging Face login failed")

# Install required libraries
!pip install langchain transformers bitsandbytes langchain-huggingface

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from langchain.prompts import PromptTemplate
from langchain_huggingface import HuggingFacePipeline
from langchain.agents import initialize_agent, Tool
from langchain.agents.agent_types import AgentType
import torch

# Define BitsAndBytesConfig for 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load the local LLM with 4-bit quantization
model_id = "mistralai/Mistral-7B-Instruct-v0.3"
llm = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"": 0})
tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side='left', add_eos_token=True, use_fast=False)

# Create a text generation pipeline
pipe = pipeline("text-generation", model=llm, tokenizer=tokenizer)

# Wrap the pipeline for LangChain
common_params = {
  'max_length': 512,
  'eos_token_id': tokenizer.eos_token_id,
}

pipelineBase = pipeline(
    "text-generation",
    model=llm,
    tokenizer=tokenizer,
    **common_params,
    max_new_tokens=200
)

llmPipeline = HuggingFacePipeline(pipeline=pipelineBase, model_kwargs={"temperature": 0.1})

# Define the prompt template
prompt_template = """
<s>
[INST]
Below is an instruction that describes a task. Write a response that appropriately completes the request.
{query}
[/INST]
</s>
[INST]Keep your response succinct.[/INST]
"""

PROMPT = PromptTemplate.from_template(prompt_template)

# Define tools
def call_llm(input_text):
    PROMPT = PromptTemplate.from_template(prompt_template)
    llm_chain = PROMPT | llmPipeline
    result = llm_chain.invoke({"query": input_text})
    return result

local_llm = llmPipeline

tools = [
    Tool(
        name="Mistral",
        func=call_llm,
        description="This tool generates a response using the local LLM pipeline and a custom prompt template."
    )
]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
logged in to Hugging Face


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0
Device set to use cuda:0


In [4]:
# Create a text generation pipeline
pipe = pipeline("text-generation", model=llm, tokenizer=tokenizer)

# Wrap the pipeline for LangChain
common_params = {
  'max_length': 512,
  'eos_token_id': tokenizer.eos_token_id,
}

pipelineBase = pipeline(
    "text-generation",
    model=llm,
    tokenizer=tokenizer,
    **common_params,
    max_new_tokens=200
)

llmPipeline = HuggingFacePipeline(pipeline=pipelineBase, model_kwargs={"temperature": 0.1})


Device set to use cuda:0
Device set to use cuda:0


In [13]:

# Define the prompt template
prompt_template = """
<s>
   [INST]
   Below is an instruction that describes a task. Write a response that appropriately completes the request.
   If you have Final Answer, leave Action/Action Input/Observation blank.
   {query}
   [/INST]
   </s>
   [INST]
   Use the following format:
    Thought: [Your thought process]
    Action: [The action to take]
    Action Input: [Input for the action]
    Observation: [Result of the action]
    Final Answer: [The final answer to the query]
   [/INST]
"""

PROMPT = PromptTemplate.from_template(prompt_template)

# Define tools
def call_llm(input_text):
    PROMPT = PromptTemplate.from_template(prompt_template)
    llm_chain = PROMPT | llmPipeline
    result = llm_chain.invoke({"query": input_text})
    return result

local_llm = llmPipeline

tools = [
    Tool(
        name="Mistral",
        func=call_llm,
        description="This tool generates a response using the local LLM pipeline and a custom prompt template."
    )
]

from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.exceptions import OutputParserException
from langchain_core.output_parsers import BaseOutputParser

class CustomOutputParser(BaseOutputParser):
    def parse(self, text: str):
        if "Final Answer:" in text:
            # Handle the final answer
            return AgentFinish(return_values={"output": text.split("Final Answer:")[-1].strip()}, log=text)
        elif "Action:" in text and "Action Input:" in text:
            # Handle action parsing
            action = text.split("Action:")[-1].split("Action Input:")[0].strip()
            action_input = text.split("Action Input:")[-1].strip()
            return AgentAction(tool=action, tool_input=action_input, log=text)
        else:
            # Raise an exception if the format isn't correct
            raise OutputParserException("Could not parse LLM output properly.")


# Initialize the agent
agent = initialize_agent(
    tools,
    local_llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=3,
    output_parser=CustomOutputParser()
)

def main():

    while True:
        user_input = input("Enter query:").lower()

        if user_input == 0:
            print("Exiting.")
            break

        # Run the agent
        result = agent.run(PROMPT.format(query=user_input))
        print(result)

if __name__ == "__main__":
    main()

Enter query:sort the following list into alphabetic order: D,A,B,N,K


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=200) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)




> Entering new AgentExecutor chain...


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=200) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Parsing LLM output produced both a final answer and a parse-able action:: Answer the following questions as best you can. You have access to the following tools:

Mistral(input_text) - This tool generates a response using the local LLM pipeline and a custom prompt template.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Mistral]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: 
<s>
   [INST]
   Below is an instruction that describes a task. Write a response that appropriately completes the request. 
   If you have Final Answer, leave Action/Action Input/Observation blank.
   sort the following list into alphabetic order: d,a,b,n,k
   [/INST]
   </s>
   [I

...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=200) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Parsing LLM output produced both a final answer and a parse-able action:: Answer the following questions as best you can. You have access to the following tools:

Mistral(input_text) - This tool generates a response using the local LLM pipeline and a custom prompt template.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Mistral]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: 
<s>
   [INST]
   Below is an instruction that describes a task. Write a response that appropriately completes the request. 
   If you have Final Answer, leave Action/Action Input/Observation blank.
   sort the following numbers into ascending values: 9,4,2,6,7,4,1,5,6
   [/INST]
  

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=200) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Parsing LLM output produced both a final answer and a parse-able action:: Answer the following questions as best you can. You have access to the following tools:

Mistral(input_text) - This tool generates a response using the local LLM pipeline and a custom prompt template.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Mistral]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: 
<s>
   [INST]
   Below is an instruction that describes a task. Write a response that appropriately completes the request. 
   If you have Final Answer, leave Action/Action Input/Observation blank.
   sort the following numbers into ascending values: 9,4,2,6,7,4,1,5,6
   [/INST]
  

KeyboardInterrupt: Interrupted by user